In [ ]:
!pip install ultralytics
!pip install streamlit
!pip install opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import cv2
import tempfile
import os
from ultralytics import YOLO
from PIL import Image
import numpy as np
from google.colab import files

# Load mô hình YOLO đã huấn luyện
model = YOLO("/content/best.pt")

st.sidebar.title("📌 Metrics")

# Sử dụng expander để ẩn/hiện bảng khi click
with st.sidebar.expander("📊 1. Bảng kết quả đánh giá hiệu suất trên tập test"):
    data = {
        "Class": ["all", "FallDown", "Not_Fallen"],
        "Images": [4788, 3488, 1303],
        "Instances": [4807, 3490, 1317],
        "Precision (P)": [0.973, 0.952, 0.994],
        "Recall (R)": [0.972, 0.950, 0.994],
        "mAP@0.5": [0.989, 0.984, 0.994],
        "mAP@0.5:0.95": [0.860, 0.818, 0.902]
    }
    df_metrics = pd.DataFrame(data)
    st.dataframe(df_metrics, use_container_width=True)

with st.sidebar.expander("📈 2. Precision"):
    st.image("/content/P_curve.png", caption="Biểu đồ Precision", use_container_width=True)


with st.sidebar.expander("📉 3. Recall"):
    st.image("/content/R_curve.png", caption="Biểu đồ Recall", use_container_width=True)

# 4. F1-Score
with st.sidebar.expander("🎯 4. F1-Score"):
    st.image("/content/F1_curve.png", caption="Biểu đồ F1-Score", use_container_width=True)

uploaded_video = st.file_uploader("📥 Upload video để phát hiện té ngã", type=["mp4"])

if uploaded_video is not None:
    tfile = tempfile.NamedTemporaryFile(delete=False)
    tfile.write(uploaded_video.read())
    video_path = tfile.name

    # Tạo đường dẫn video output
    output_path = os.path.join(tempfile.gettempdir(), "output.mp4")

    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # VideoWriter để ghi lại video xử lý
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    stframe = st.empty()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]
        annotated_frame = results.plot()

        out.write(annotated_frame)  # Ghi frame đã xử lý
        stframe.image(annotated_frame, channels="BGR", use_container_width=True)

    cap.release()
    out.release()
    st.success("🎉 Xử lý video hoàn tất!")

    # Cho phép người dùng tải về video
    with open(output_path, "rb") as f:
        video_bytes = f.read()
        st.download_button(
            label="📥 Tải video đã xử lý",
            data=video_bytes,
            file_name="fall_detected_output.mp4",
            mime="video/mp4"
        )


Writing app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.16.157.82


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.90.212.155:8501

your url is: https://four-bananas-dream.loca.lt
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
2025-05-06 06:50:23.648 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-p